# Load Data

In [ ]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

In [ ]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# ANN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Load supplier data
supplier_data = pd.read_csv("Supplier_Data.csv")

# Define the custom loss function with penalties for violating constraints
def custom_loss(y_true, y_pred):
    # Calculate the base loss
    base_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

    # Calculate the penalty for exceeding capacity
    capacity_penalty = tf.maximum(0.0, y_pred - supplier_data["Capacity"])

    # Calculate the penalty for not meeting demand
    demand_penalty = tf.maximum(0.0, y_true - tf.reduce_sum(y_pred))

    # Calculate the cost of the orders
    order_cost = tf.reduce_sum(y_pred * supplier_data["Price"])

    # Combine the base loss with the penalties and the cost
    total_loss = base_loss + tf.reduce_sum(capacity_penalty) + demand_penalty + order_cost

    return total_loss

# Define the ANN model
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='linear'))  # 10 suppliers

# Compile the model with the custom loss function
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=custom_loss)

# Train the model
# X_train, y_train should be prepared beforehand
model.fit(data_train, target_train, epochs=10)

# Predict the demand allocation
y_pred = model.predict(data_test)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'learning_rate': 0.0014629856227712718, 'n_hidden': 3, 'n_neurons': 79}
MSE: 21420.868463572395


# DT

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': [10, 20, 30, 40, 50],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'feature_fraction': [0.8, 0.9, 1.0],
    'bagging_fraction': [0.8, 0.9, 1.0],
    'bagging_freq': [3, 4, 5, 6, 7],
}

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=critical_ratio, metric='quantile', verbose=0)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(data_train, target_train)
print(rnd_search.best_params_)

# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(data_test)
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\lightgbm\sklearn.py", line 1092, in fit
    super().fit(
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\lightgbm\sklearn.

# Costs

In [87]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
overall_costs_ANN = 0
overall_costs_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * np.round(target_pred_ANN[i])
    profit_DT = (p - c) * np.round(target_pred_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if target_pred_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - np.round(target_pred_ANN[i]))
    if target_pred_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (np.round(target_pred_ANN[i]) - target_test.values[i])
    if target_pred_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - np.round(target_pred_DT[i]))
    if target_pred_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (np.round(target_pred_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT

    # Calculate the total costs for the week
    total_costs_ANN = overstock_ANN + understock_ANN
    overall_costs_ANN += total_costs_ANN
    total_costs_DT = overstock_DT + understock_DT
    overall_costs_DT += total_costs_DT
   

# Print the overall profit
print('Overall profit for ANN: ', int(overall_profit_ANN))
print('Overall costs for ANN: ', int(overall_costs_ANN))
print('Overall profit for DT: ', int(overall_profit_DT))
print('Overall costs for DT: ', int(overall_costs_DT))

Overall profit for ANN:  70031
Overall costs for ANN:  15484
Overall profit for DT:  79009
Overall costs for DT:  5510


C:\Users\lanza\AppData\Local\Temp\ipykernel_560\3253016254.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overall profit for ANN: ', int(overall_profit_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_560\3253016254.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overall costs for ANN: ', int(overall_costs_ANN))
C:\Users\lanza\AppData\Local\Temp\ipykernel_560\3253016254.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Overall pr